# Exercício 07_03

#### Juliana Carolina Thuler dos Santos

## Exploração

Utilizando o conjunto de dados de compras (`compra_online_treino.csv`), testar qual modelo (Regressão Logística, KNN e Árvore) se adaptará melhor a este conjunto de dados. Realizar o pré-processamento necessário para cada um, explicando o por quê desta decisão.

PS: apesar do nomes do arquivo ser `compra_online_treino.csv`, separar em treino e teste, aplicando a metodologia que você julgar melhor.

In [43]:
import numpy as np
import pandas as pd

df = pd.read_csv('compra_online_treino.csv').iloc[:,1:]

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9247 entries, 0 to 9246
Data columns (total 18 columns):
Administrative             9173 non-null float64
Administrative_Duration    9182 non-null float64
Informational              9171 non-null float64
Informational_Duration     9170 non-null float64
ProductRelated             9173 non-null float64
ProductRelated_Duration    9164 non-null float64
BounceRates                9168 non-null float64
ExitRates                  9166 non-null float64
PageValues                 9177 non-null float64
SpecialDay                 9187 non-null float64
Month                      9174 non-null object
OperatingSystems           9181 non-null float64
Browser                    9186 non-null float64
Region                     9184 non-null float64
TrafficType                9193 non-null float64
VisitorType                8075 non-null object
Weekend                    9167 non-null float64
Revenue                    9247 non-null bool
dtypes: bool(1),

In [134]:
numericas = ['Administrative', 'Administrative_Duration', 'Informational',
             'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
             'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']

categoricas_num = ['OperatingSystems', 'Browser', 'Region', 'TrafficType', 'Weekend']

categoricas_qlt = ['Month','VisitorType']

In [135]:
from sklearn.model_selection import train_test_split as tts

df_treino, df_teste = tts(df, test_size = 0.2, random_state = 42)

#### Regressão Logística

In [197]:
df_rl_treino = df_treino.copy()
df_rl_teste = df_teste.copy()

In [198]:
#for col in numericas: #talvez seja uma informação útil
#    print(len(df_rl_treino[df_rl_treino[col] == -1]))

In [199]:
#df_rl_treino.isnull().sum()

In [200]:
#df_rl_teste.isnull().sum()
#df_rl_teste.info()

In [201]:
#substituir as numéricas faltantes com a média
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = 'median')
df_rl_treino[numericas] = si.fit_transform(df_rl_treino[numericas])
df_rl_teste[numericas] = si.transform(df_rl_teste[numericas])

In [202]:
#substituir as categoricas numericas faltantes com a moda
si_num = SimpleImputer(strategy = 'most_frequent')

df_rl_treino[categoricas_num] = si_num.fit_transform(df_rl_treino[categoricas_num])
df_rl_teste[categoricas_num] = si_num.transform(df_rl_teste[categoricas_num])

In [203]:
#substituir as categoricas qualitativas faltantes com valor diferente de qualquer um da amostra
si_qlt = SimpleImputer(strategy = 'constant', fill_value = 'Missing')

df_rl_treino[categoricas_qlt] = si_qlt.fit_transform(df_rl_treino[categoricas_qlt])
df_rl_teste[categoricas_qlt] = si_qlt.transform(df_rl_teste[categoricas_qlt])

In [204]:
#transformar as categoricas qualitativas em numericas
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()

df_rl_treino[categoricas_qlt] = oe.fit_transform(df_rl_treino[categoricas_qlt])
df_rl_teste[categoricas_qlt] = oe.transform(df_rl_teste[categoricas_qlt])

In [205]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(df_rl_treino.drop(columns = 'Revenue'),df_rl_treino[['Revenue']])

c:\users\jtsantos\my-venv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jtsantos\my-venv\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [206]:
x_teste = df_rl_teste.drop(columns = 'Revenue')
y_teste = df_rl_teste[['Revenue']]

pred = lr.predict(x_teste)

In [207]:
from sklearn.metrics import accuracy_score as asc
round(asc(pred,y_teste),2)

0.97

In [208]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_teste['Revenue'].values, pred).ravel()
print(tn, fp, fn, tp)

1547 18 42 243


#### Considerações

Como sklearn não considera variáveis faltantes ou do tipo texto, fiz as transformações necessárias.
Para as numéricas faltantes considerei a mediana (uma vez que os valores máximos apresentavam-se muito distante da mediana, o que com certeza afetaria a média), e para as categóricas numéricas faltantes considerei a moda. 

As categóricas qualitativas, por apresentarem um número muito grande de faltantes, preferi transformar as faltantes em um valor diferente. Após tal transformação, transformei em numéricas. 

Pelos cálculos matemáticos envolvidos na regressão logística, considerando o número de euler elevado a um valor negativo, e o valor em questão considerar fatores de multiplicação em função da importância das variáveis,~não acreditei ser necessário mudar as escalas nos valores numéricos. 

#### KNN

In [230]:
#necessitaria realizar todas as transformações já realizadas até então nos df's para Regressão Logística
df_knn_treino = df_rl_treino.copy()
df_knn_teste = df_rl_teste.copy()

In [231]:
#realização de knn sem normalizar as colunas de variáveis numéricas e categoricas numericas
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(df_knn_treino.drop(columns = 'Revenue'),df_knn_treino[['Revenue']])

c:\users\jtsantos\my-venv\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [232]:
x_teste = df_knn_teste.drop(columns = 'Revenue')
y_teste = df_knn_teste[['Revenue']]

pred = knn.predict(x_teste)

In [233]:
#acurácia
round(asc(pred,y_teste),2)

0.87

In [234]:
#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_teste['Revenue'].values, pred).ravel()
print(tn, fp, fn, tp)

1518 47 199 86


In [235]:
#normalizar as colunas de variáveis numéricas e categoricas numericas
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
df_knn_treino[numericas] = mms.fit_transform(df_knn_treino[numericas])
df_knn_teste[numericas] = mms.transform(df_knn_teste[numericas])

df_knn_treino[categoricas_num] = mms.fit_transform(df_knn_treino[categoricas_num])
df_knn_teste[categoricas_num] = mms.transform(df_knn_teste[categoricas_num])

In [236]:
knn.fit(df_knn_treino.drop(columns = 'Revenue'),df_knn_treino[['Revenue']])

c:\users\jtsantos\my-venv\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [237]:
x_teste = df_knn_teste.drop(columns = 'Revenue')
y_teste = df_knn_teste[['Revenue']]

pred = knn.predict(x_teste)

In [238]:
#acurácia
round(asc(pred,y_teste),2)

0.96

In [239]:
#confusion matrix
tn, fp, fn, tp = confusion_matrix(y_teste['Revenue'].values, pred).ravel()
print(tn, fp, fn, tp)

1555 10 57 228


##### Considerações

Nesse caso, pudemos perceber a importância de normalizar as colunas da amostra uma vez que o KNN considera uma equeação de "distância entre pontos" e, nesse caso, as amplitudes podem alterar sigficativamente nossos resultados.

#### Árvore de Decisão

In [240]:
#considerar as manipulações realizadas para a regressão logística
df_add_treino = df_rl_treino.copy()
df_add_teste = df_rl_teste.copy()

In [283]:
from sklearn.tree import DecisionTreeClassifier

for i in np.arange(3,9,1):
    
    dt = DecisionTreeClassifier(max_depth = i)
    dt.fit(df_add_treino.drop(columns = 'Revenue'),df_add_treino[['Revenue']])
    
    x_teste = df_add_teste.drop(columns = 'Revenue')
    y_teste = df_add_teste[['Revenue']]
    
    pred = dt.predict(x_teste)
    
    tn, fp, fn, tp = confusion_matrix(y_teste['Revenue'].values, pred).ravel()
    
    print('Profundidade: ', i)
    print('Ácurácia: ',round(asc(pred,y_teste),2))
    print('TN, FP, FN, TP: ', tn, fp, fn, tp)
    print('\n')

Profundidade:  3
Ácurácia:  0.97
TN, FP, FN, TP:  1547 18 46 239


Profundidade:  4
Ácurácia:  0.97
TN, FP, FN, TP:  1547 18 40 245


Profundidade:  5
Ácurácia:  0.97
TN, FP, FN, TP:  1553 12 43 242


Profundidade:  6
Ácurácia:  0.96
TN, FP, FN, TP:  1545 20 46 239


Profundidade:  7
Ácurácia:  0.96
TN, FP, FN, TP:  1543 22 50 235


Profundidade:  8
Ácurácia:  0.95
TN, FP, FN, TP:  1539 26 58 227




In [263]:
#realizando PCA para selecionar as principais componentes e talvez excluir a possibilidade de overfitting
from sklearn.decomposition import PCA

x_treino = df_add_treino.drop(columns = ['Revenue'])
y_treino = df_add_treino[['Revenue']]

pca = PCA()
pca.fit_transform(x_treino)

pca.explained_variance_ratio_

array([9.89219542e-01, 6.54630043e-03, 4.00081591e-03, 1.39040494e-04,
       8.38347119e-05, 4.22005099e-06, 1.81958735e-06, 1.53138397e-06,
       1.43512582e-06, 7.39683713e-07, 2.64865019e-07, 2.09086322e-07,
       1.92214204e-07, 4.40421559e-08, 9.90664399e-09, 9.48551640e-10,
       5.00661969e-11])

In [286]:
#vou considerar as 5 primeiras, considerando a ordem de grandeza das variâncias explicadas
pca = PCA(n_components = (len(df_add_treino.columns)//3))
pca.fit(x_treino)
x_treino_transf = pca.transform(x_treino)

x_teste = df_add_teste.drop(columns = ['Revenue'])
y_teste = df_add_teste[['Revenue']]

x_teste_transf = pca.transform(x_teste)

In [287]:
for i in np.arange(3,9,1):
    
    dt = DecisionTreeClassifier(max_depth = i)
    dt.fit(x_treino_transf,df_add_treino[['Revenue']])

    y_teste = df_add_teste[['Revenue']]
    
    pred = dt.predict(x_teste_transf)
    
    tn, fp, fn, tp = confusion_matrix(y_teste['Revenue'].values, pred).ravel()
    
    print('Profundidade: ', i)
    print('Ácurácia: ',round(asc(pred,y_teste),2))
    print('TN, FP, FN, TP: ', tn, fp, fn, tp)
    print('\n')

Profundidade:  3
Ácurácia:  0.89
TN, FP, FN, TP:  1433 132 80 205


Profundidade:  4
Ácurácia:  0.89
TN, FP, FN, TP:  1522 43 165 120


Profundidade:  5
Ácurácia:  0.89
TN, FP, FN, TP:  1495 70 134 151


Profundidade:  6
Ácurácia:  0.89
TN, FP, FN, TP:  1486 79 129 156


Profundidade:  7
Ácurácia:  0.88
TN, FP, FN, TP:  1478 87 130 155


Profundidade:  8
Ácurácia:  0.88
TN, FP, FN, TP:  1486 79 147 138




#### Considerações

Neste caso, esperava-se que com o método PCA a assertividade do modelo fosse maior, porém, tivemos o contrário. 
Ainda assim, os resultados são satisfatórios. 


### Considerações finais

Para este dataset, os três modelos são eficientes, desde que realizemos as transformações adequadas.